## Import dataset 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re

In [2]:
plt.style.use('ggplot')
%matplotlib inline

In [3]:
df_all = pd.read_csv('data/april_report.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_all.columns = [col.lower().replace(' ','_') for col in df_all.columns.tolist()]
df_all.columns = [col.lower().replace('/timestamp','') for col in df_all.columns.tolist()]

In [5]:
date_col = [col for col in df_all.columns.tolist() if 'date' in col]
df_all[date_col] = df_all[date_col].apply(pd.to_datetime,format = "%m/%d/%Y %H:%M:%S")

In [6]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253306 entries, 0 to 1253305
Data columns (total 23 columns):
unique_row_id           1253306 non-null object
cvo_sid                 1253306 non-null object
event_site_id           1253306 non-null int64
touchpoint_site_id      1253306 non-null int64
event_site_name         1253306 non-null object
touchpoint_site_name    1253306 non-null object
event_type              1253306 non-null object
event_id                1253306 non-null object
event_date              1253306 non-null datetime64[ns]
touchpoint_date         1253306 non-null datetime64[ns]
total_event_value       1253306 non-null float64
touchpoint_revenue      1253306 non-null float64
conversion              1253306 non-null float64
touchpoint_type         1027589 non-null object
first_touch_date        1253306 non-null datetime64[ns]
last_touch_date         1253306 non-null datetime64[ns]
source_1                1253306 non-null object
source_2                1253306 non-nu

## Clean
* pick columns
* remove channel base


In [7]:
selected_col=['event_site_name','touchpoint_site_name','event_type','conversion','touchpoint_type','source_1','touchpoints_in_path','attribution_sequence','path_position']
selected_col.extend(date_col)

df_prep=df_all[selected_col]

In [8]:
df_prep = df_prep[df_prep.source_1!='base']
# df_prep = df_prep[df_prep.touchpoints_in_path<50]
df_prep.reset_index(drop=True, inplace=True)
df_prep.loc[df_prep.source_1.str.contains('social'),'source_1']='social'

In [9]:
def remove_small_category(df,col):
    count = df[col].value_counts()
    index = count[count<1000].index.tolist()
    return df[~df[col].isin(index)]

In [10]:
df_prep = remove_small_category(df_prep,'source_1')
df_prep = remove_small_category(df_prep,'event_type')

In [11]:
df_prep.head()

,event_site_name,event_type,conversion,touchpoint_type,source_1,touchpoints_in_path,attribution_sequence,path_position,event_date,touchpoint_date,first_touch_date,last_touch_date
0,Nexium24hour,nexium24hour-buy.now,1.000000,visit,ppc nonbrand,1,1,only,2017-04-07 17:54:58,2017-04-07 17:54:58,2017-04-07 17:54:58,2017-04-07 17:54:58
1,Centrum,centrum-coupon.submit,0.000047,visit,organic not provided,2,1,introducer,2017-04-21 17:52:51,2017-04-12 12:26:32,2017-04-12 12:26:32,2017-04-21 17:52:51
2,Centrum,centrum-coupon.submit,0.011891,visit,organic not provided,2,2,closer,2017-04-21 17:52:51,2017-04-21 17:50:56,2017-04-12 12:26:32,2017-04-21 17:52:51
3,Centrum,centrum-coupon.submit,0.197314,visit,ppc brand,1,1,only,2017-04-30 18:04:52,2017-04-30 18:04:22,2017-04-30 18:04:22,2017-04-30 18:04:52
4,Centrum,centrum-email.sign.up,1.000000,visit,social,1,1,only,2017-04-19 14:12:59,2017-04-19 14:12:00,2017-04-19 14:12:00,2017-04-19 14:12:00


In [12]:
df_prep.source_1.value_counts()

display                 726559
video                   164456
ppc brand                36791
organic not provided     22399
content                  18998
other                    15348
social                   14918
coupon                   11951
ppc nonbrand              7640
email                     6023
organic brand             1452
Name: source_1, dtype: int64

In [13]:
df_prep.shape

(1026535, 12)

## procesing
* conbime servel social
* days from first to last touch
* distance to event
* mean of __event, source, path distant, path position, touch point type__

In [13]:
df_prep['event_to_touchpoint']=(df_prep.event_date-df_prep.touchpoint_date).dt.days

df_prep['day_of_week']=df_prep.touchpoint_date.dt.weekday

df_prep['month']=df_prep.touchpoint_date.dt.month

df_prep['week_of_month']=(df_prep.touchpoint_date.dt.day-1)//7+1

df_prep['distance_event'] = df_prep.touchpoints_in_path - df_prep.attribution_sequence

In [14]:
df_prep.head()

,event_site_name,event_type,conversion,touchpoint_type,source_1,touchpoints_in_path,attribution_sequence,path_position,event_date,touchpoint_date,first_touch_date,last_touch_date,event_to_touchpoint,day_of_week,month,week_of_month,distance_event
0,Nexium24hour,nexium24hour-buy.now,1.000000,visit,ppc nonbrand,1,1,only,2017-04-07 17:54:58,2017-04-07 17:54:58,2017-04-07 17:54:58,2017-04-07 17:54:58,0,4,4,1,0
1,Centrum,centrum-coupon.submit,0.000047,visit,organic not provided,2,1,introducer,2017-04-21 17:52:51,2017-04-12 12:26:32,2017-04-12 12:26:32,2017-04-21 17:52:51,9,2,4,2,1
2,Centrum,centrum-coupon.submit,0.011891,visit,organic not provided,2,2,closer,2017-04-21 17:52:51,2017-04-21 17:50:56,2017-04-12 12:26:32,2017-04-21 17:52:51,0,4,4,3,0
3,Centrum,centrum-coupon.submit,0.197314,visit,ppc brand,1,1,only,2017-04-30 18:04:52,2017-04-30 18:04:22,2017-04-30 18:04:22,2017-04-30 18:04:52,0,6,4,5,0
4,Centrum,centrum-email.sign.up,1.000000,visit,social,1,1,only,2017-04-19 14:12:59,2017-04-19 14:12:00,2017-04-19 14:12:00,2017-04-19 14:12:00,0,2,4,3,0


In [15]:
df_prep['round_conversion'] = df_prep.conversion.round(decimals=4)

In [67]:
df_prep.conversion.quantile(np.arange(0,1,0.05))

0.00    0.000000
0.05    0.000000
0.10    0.000000
0.15    0.000001
0.20    0.000002
0.25    0.000004
0.30    0.000008
0.35    0.000031
0.40    0.000164
0.45    0.000371
0.50    0.000648
0.55    0.000986
0.60    0.001451
0.65    0.002141
0.70    0.003176
0.75    0.004795
0.80    0.007491
0.85    0.013747
0.90    0.040693
0.95    0.218315
Name: conversion, dtype: float64

add feature mean

 Get Dummies

In [16]:
df_prep_dummies=pd.get_dummies(df_prep,columns=['event_site_name','event_type','touchpoint_type','source_1','path_position'],
                          drop_first=True)

In [17]:
#drop column with date
df_model=df_prep_dummies.drop(date_col,axis=1)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
label = df_model.round_conversion
feature = df_model.drop(['conversion','round_conversion'],axis=1)

In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
label = le.fit_transform(label)

In [22]:
df_train,df_test,label_train,label_test = train_test_split(feature,label,test_size=.3,random_state=0)

## modeling

In [23]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [24]:
from sklearn.naive_bayes import GaussianNB

In [25]:
clf = GaussianNB()
clf.fit(df_train,label_train)

GaussianNB(priors=None)

In [38]:
pred = clf.predict(df_test.iloc[0:100,:])

In [44]:
a1 = le.inverse_transform(pred)

In [45]:
a2=le.inverse_transform(label_test[:100])

In [50]:
metrics.r2_score(a2,a1)

-0.49339174175748424

In [43]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import (Ridge, TheilSenRegressor, RANSACRegressor, HuberRegressor)

from sklearn.ensemble import (RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor,
                              AdaBoostRegressor, GradientBoostingRegressor)

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn import metrics

* random forest

In [ ]:
rf = RandomForestRegressor(n_estimators=50,max_depth=10,random_state=0)

In [ ]:
rf.fit(df_train,label_train)

In [ ]:
rf.score(df_test,label_test)

In [ ]:
predict1 = rf.predict(df_test)

metrics.mean_squared_error(label_test,predict1)

* knn

In [ ]:
knn = KNeighborsRegressor(n_neighbors=10)

In [ ]:
knn.fit(df_train,label_train)

In [ ]:
knn.score(df_test,label_test)

* elstic net

In [ ]:
eln = ElasticNet(alpha=0.05)

In [ ]:
eln.fit(df_train,label_train)

In [ ]:
eln.score(df_test,label_test)

* grainting boosting

In [ ]:
gbr = GradientBoostingRegressor(random_state=0)

In [ ]:
gbr.fit(df_train,label_train)

In [ ]:
gbr.score(df_test,label_test)

* bagging regressor

In [ ]:
bag = BaggingRegressor(random_state=0)

In [ ]:
bag.fit(df_train,label_train)

In [ ]:
bag.score(df_test,label_test)

In [ ]:
predict1 = bag.predict(df_test)

metrics.mean_squared_error(label_test,predict1)